# 상수, 변수, 플레이스홀더

In [2]:
import tensorflow as tf

## 상수 (tf.constant)

``` py
tf.constant(
    value,
    dtype=None,
    shape=None,
    name='Const',
    verify_shape=False
)
```

텐서플로에서 상수는 고정값을 의미합니다. 값 자체는 행렬이거나 텐서여도 무관합니다.

행렬이나 텐서를 상수값으로 넣을때는 주로 넘피(numpy) 데이터형을 사용하게 됩니다. 이때 shape을 굳이 넣지 않아도 텐서플로는 자연스럽게 np.shape 값을 shape 값으로 기록하게 됩니다.

만약 shape값을 직접 지정해주었지만, value 행렬의 크기가 더 클경우에는 오류를 반환합니다.
반대로, 지정된 shape값보다 value 행렬 크기가 작을 경우, 남은 공간들을 value에 가장 마지막 값으로 복사해 넣게됩니다.
verify_shape이 참(true)일 경우에는 shape값과 value의 행렬크기가 같지 않는 경우 모두 오류로 처리됩니다.

In [28]:
tf.reset_default_graph()

x = tf.constant([1,4,3], shape=(3,3), name='x')
y = tf.constant(5, name='y')
z = tf.constant([[1,3],[2,5]])

sess = tf.Session()
print(x, '\n', sess.run(x))
print(y, '\n', sess.run(y))
print(z, '\n', sess.run(z))
sess.close()

Tensor("x:0", shape=(3, 3), dtype=int32) 
 [[1 4 3]
 [3 3 3]
 [3 3 3]]
Tensor("y:0", shape=(), dtype=int32) 
 5
Tensor("Const:0", shape=(2, 2), dtype=int32) 
 [[1 3]
 [2 5]]


## 플레이스홀더 (tf.placeholder)

``` py
tf.placeholder(
    dtype,
    shape=None,
    name=None
)
```

플레이스홀더는 계산중 입력값을 받는 변수입니다. 일반 변수와의 차이점은 플레이스홀더는 실행당시 직접 값을 넣어주어야 하지만 변수는 항상 어떠한 값을 가지고 있습니다.

플레이스홀더를 지정할때 가장 필수적인 요소는 데이터형입니다. 데이터를 입력받을 때 어떤 데이터형으로 받을지를 미리 정해두는 것은 추후에 생길 계산에 형변환 오류를 방지하기 위해서입니다.

플레이스홀더를 지정할때 사이즈에 None값을 사용할 수 있는데, None은 크기가 정해지지 않았다는 의미를 가지고 있습니다. 만약 shape=None을 사용할 경우 그 플레이스홀더가 받아드릴 수 있는 텐서 크기에 제약이 없습니다. 물론 텐서가 어떻게 진행되고 텐서의 크기가 어떻게 변형되는지를 체계적으로 정리하고있다면 제약이 없어도 크게 문제는 되지 않습니다. 하지만 보통 텐서의 모양은 [None, (input shape)] 형태를 사용하는데, 이러한 경우 데이터가 어떤 형태로 들어오는지를 미리 지정함으로써 추후에 계산을 조금 더 쉽게 할수있게 도와줍니다. 앞에 None이 의미하는것은 우리가 네트워크에 넣어줄 데이터의 갯수가 일정하지 않다는것을 의미합니다. 예측을 목적으로 네트워크를 실행할때는 하나의 데이터만 실행할 수 있지만, 트레이닝을 목적으로 네트워크를 돌리는 경우는 많은 양의 데이터를 동시에 처리해야합니다.

대부분 플레이스홀더는 인풋(input)을 의미할때 사용합니다. 예를들어 인풋이 256x256 사이즈의 이미지일 경우 사이즈는 [None, 256, 256]을 사용합니다.

그래프를 실행할 때 플레이스홀더에 값을 넣는 방법은 세션을 실행하는 메소드인 run 안에 딕셔너리 형으로 넣는 방법입니다.

만약 입력된 데이터 값과 플레이스홀더의 shape이 맞지 않는 경우에는 오류를 반환합니다.

In [45]:
tf.reset_default_graph()

x = tf.placeholder(tf.int32, [None, 2])
print(x)

x_none = tf.placeholder(tf.int32)
print(x_none)

x_data = [[1, 2], [3, 4], [5,6]]

sess = tf.Session()
print(sess.run(x, feed_dict={x:x_data}))
print(sess.run(x_none, feed_dict={x_none:x_data}))
sess.close()

Tensor("Placeholder:0", shape=(?, 2), dtype=int32)
Tensor("Placeholder_1:0", dtype=int32)
[[1 2]
 [3 4]
 [5 6]]
[[1 2]
 [3 4]
 [5 6]]


## 변수 (tf.variable)

텐서플로 변수는 값이 변할 수 있는 텐서를 뜻합니다.

텐서플로로 구현된 그래프를 실행하기 전에 tf.global_variables_initializer() 메소드를 이용해 모든 변수를 초기값으로 초기화시킬 수 있습니다. 초기화되지 않은 변수가 사용될 때에는 오류를 반환합니다.

변수를 생성하기 위해서는 데이터형, 텐서 형태, 초기화함수, 이름 등 기본적인 내용들이 포함됩니다. 그 외에 텐서보드에 사용될 여러 파라미터나 트레이닝을 통해 변수를 변경시킬지에 대한 여부 등을 조절할 수 있습니다.

In [87]:
tf.reset_default_graph()

A = tf.Variable(tf.constant([[3,6]]), dtype=tf.int32)
x = tf.constant([[1,0],[0,2]], dtype=tf.int32)
B = tf.matmul(A,x)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(A))
print(sess.run(B))

A = A + tf.constant([[1,2]])
print(sess.run(A))
print(sess.run(B))
sess.close()

[array([[3, 6]], dtype=int32), array([[ 3, 12]], dtype=int32)]
[[4 8]]
[[ 3 12]]
